In [123]:
!pip install lxml html5lib beautifulsoup4


     -------------------------------------- 112.2/112.2 kB 6.4 MB/s eta 0:00:00


In [132]:
import pandas as pd
import sklearn 
import nltk
import numpy as np 
import matplotlib.pyplot as ply
import scipy
import re
import lxml.html
import requests

In [171]:
#functions to clean the data
def de_hashtag(row):
    temp = row['text'].split()
    res = [string for string in temp if string[0]=='#']
    return res
def websites(row):
    regex = '((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
    r = re.compile(regex)
    temp = row['text'].split()
    res = list(filter(r.match,temp))
    return res
def phone_numbers(row):
    regex = '^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$'
    r = re.compile(regex)
    temp = row['text'].split()
    res = list(filter(r.match,temp))
    if len(res)>0:
        return str(res[0])
    return np.nan
def get_geo(row):
    if not row['phone_number'].isna():
        

'def get_geo(row):\n    if len(row[])'

In [273]:
#load area codes
cols = ['area_code','city','state','country_code','lat','lng']
col_geo = ['area_code','lat','lng']
area_codes = pd.read_csv('https://raw.githubusercontent.com/ravisorg/Area-Code-Geolocation-Database/master/us-area-code-cities.csv',header=None,names=cols)
area_codes_geo = pd.read_csv('https://raw.githubusercontent.com/ravisorg/Area-Code-Geolocation-Database/master/us-area-code-geo.csv',header=None,names=cols)
area_codes.area_code= area_codes.area_code.astype(int)

#inital laod of data
df = pd.read_csv('data/ninthrun.csv',header=0)
df = df.loc[:,['text','date']]
# add hastags to extra row
df['hashtags']= df.apply(de_hashtag,axis=1)
#get unique websites
df['websites']= df.apply(websites,axis=1)
# phone numbers
df['phone_numbers']= df.apply(phone_numbers,axis=1)
# add unique count to each set of tweets
df['unique_count']=df.groupby('text')['date'].rank(method='first')
df['area_code'] = df.phone_numbers.str[:3]
df['area_code'].fillna('999999999',inplace=True)
df.area_code = df.area_code.astype(int)
df= df.merge(area_codes[['area_code','state','country_code']],left_on='area_code',right_on='area_code',how='left')
df.area_code = df.area_code.astype(object)
df.replace({'area_code':999999999},{'area_code':np.nan},regex=True,inplace=True)
df.head()

,text,date,hashtags,websites,phone_numbers,unique_count,area_code,state,country_code
0,"To a GREAT Friend &amp; Kind Heart, here are A...",2020-12-07 15:57:29+00:00,"[#WakeUpCall, #FloydLittle, #Little, #Syracuse...","[https://t.co/Ho2RA8efJ2, https://t.co/1RrhcB4...",NaN,1.0,NaN,NaN,NaN
1,remember when #Syracuse was a football blue bl...,2020-10-20 22:46:37+00:00,"[#Syracuse, #brown, #mcnabb, #graves, #morris,...",[https://t.co/qzZS0jwaG4],NaN,1.0,NaN,NaN,NaN
2,#Syracuse #orange #autograph #floyd #little #j...,2020-04-18 08:39:16+00:00,"[#Syracuse, #orange, #autograph, #floyd, #litt...",[https://t.co/40pmQZGC8j],NaN,1.0,NaN,NaN,NaN
3,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 14:28:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,28.0,NaN,NaN,NaN
4,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 05:15:17+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,19.0,NaN,NaN,NaN


In [268]:
df[~df['area_code'].isna()]

,text,date,hashtags,websites,phone_numbers,unique_count,area_code,state,country_code
166,"In Syracuse, NY now til noon Tomorrow. On east...",2018-08-08 18:34:52+00:00,"[#Escort, #Syracuse, #Cuse, #ErosNYC, #erosgir...",[],917-406-4563,1.0,917.0,New York,US
167,"In Syracuse, NY now til noon Tomorrow. On east...",2018-08-08 18:34:52+00:00,"[#Escort, #Syracuse, #Cuse, #ErosNYC, #erosgir...",[],917-406-4563,1.0,917.0,New York,US
168,"In Syracuse, NY now til noon Tomorrow. On east...",2018-08-08 18:34:52+00:00,"[#Escort, #Syracuse, #Cuse, #ErosNYC, #erosgir...",[],917-406-4563,1.0,917.0,New York,US
169,"In Syracuse, NY now til noon Tomorrow. On east...",2018-08-08 18:34:52+00:00,"[#Escort, #Syracuse, #Cuse, #ErosNYC, #erosgir...",[],917-406-4563,1.0,917.0,New York,US
170,"In Syracuse, NY now til noon Tomorrow. On east...",2018-08-08 18:34:52+00:00,"[#Escort, #Syracuse, #Cuse, #ErosNYC, #erosgir...",[],917-406-4563,1.0,917.0,New York,US
...,...,...,...,...,...,...,...,...,...
19178,#Available Now in #Albany #NY \n**Accepting Ne...,2021-07-20 16:28:48+00:00,"[#Available, #Albany, #NY, #booking, #incall, ...",[https://t.co/jLKuNhLnwT],518-299-1848,3.0,518.0,New York,US
19179,#Available Now in #Albany #NY \n**Accepting Ne...,2021-07-20 16:28:48+00:00,"[#Available, #Albany, #NY, #booking, #incall, ...",[https://t.co/jLKuNhLnwT],518-299-1848,3.0,518.0,New York,US
19180,#Available Now in #Albany #NY \n**Accepting Ne...,2021-07-20 16:28:48+00:00,"[#Available, #Albany, #NY, #booking, #incall, ...",[https://t.co/jLKuNhLnwT],518-299-1848,3.0,518.0,New York,US
19181,#Available Now in #Albany #NY \n**Accepting Ne...,2021-07-20 16:28:48+00:00,"[#Available, #Albany, #NY, #booking, #incall, ...",[https://t.co/jLKuNhLnwT],518-299-1848,3.0,518.0,New York,US


In [272]:
area_codes

,area_code,city,state,country_code,lat,lng
0,201,Bayonne,New Jersey,US,40.66871,-74.11431
1,201,Bergenfield,New Jersey,US,40.92760,-73.99736
2,201,Cliffside Park,New Jersey,US,40.82149,-73.98764
3,201,Englewood,New Jersey,US,40.89288,-73.97264
4,201,Fair Lawn,New Jersey,US,40.94038,-74.13181
...,...,...,...,...,...,...
2761,854,Hilton Head Island,South Carolina,US,32.21632,-80.75261
2762,854,Mount Pleasant,South Carolina,US,32.79407,-79.86259
2763,854,North Myrtle Beach,South Carolina,US,33.81601,-78.68002
2764,854,North Charleston,South Carolina,US,32.85462,-79.97481


In [271]:
df['state'].value_counts()

Florida          860
New York         756
Massachusetts    529
California       114
Connecticut       64
Illinois          49
Washington         9
Missouri           2
Name: state, dtype: int64

In [269]:
regex = '((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*'
r = re.compile(regex)

l = df.text.loc[8].split()
re.findall(r,df.text.loc[8])
res = list(filter(r.match,l))
res

['https://t.co/8KT13m8neo', 'https://t.co/iVrBEnIwO1']

In [270]:
df[df['text']==df['text'].loc[3]].sort_values('unique_count')

,text,date,hashtags,websites,phone_numbers,unique_count,area_code,state,country_code
6,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,1.0,NaN,NaN,NaN
2245,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,2.0,NaN,NaN,NaN
3972,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,3.0,NaN,NaN,NaN
5463,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,4.0,NaN,NaN,NaN
7190,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,5.0,NaN,NaN,NaN
8681,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,6.0,NaN,NaN,NaN
11930,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,7.0,NaN,NaN,NaN
13660,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,8.0,NaN,NaN,NaN
15389,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:38:29+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,9.0,NaN,NaN,NaN
5,RT @FloppZilla: Legends! #GOAT #SYRACUSE #Cson...,2019-04-14 01:46:01+00:00,"[#GOAT, #SYRACUSE, #Csonka, #Little, #Coughlin...",[https://t.co/OBohpJXpdM],NaN,10.0,NaN,NaN,NaN
